# One-lane comparison

In [ ]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss
import matplotlib.patches as patches

#Parámetros
N = [5,10,15,20,25,30,35,40,45] #valores de n para el experimento
pt = 1 #paso temporal
s = 100 #cantidad de simulaciones
t = 500 #este es tiempo que se usa para el movimiento de los carros
vmax = 1 #máxima velocidad que puede tener un carro
vmin= 0 #mínima velocidad
tM= 50 #tamaño del mundo

class Vehiculo:
    id_iter = itertools.count() #para crearles un id a los carros
    def __init__(self, tipo, posx, velocidad,posy):
        self.id = next(Vehiculo.id_iter) #id
        self.t = tipo #Carro para esta primera etapa
        self.x = posx #posición incial
        self.y = posy
        self.vm = velocidad #speed
        self.d = 0 #distance
        self.a = 0.01 #default value
        self.m = random.choice(["vmax", "vm"]) #change
        self.f = vmax

    def mover(self,acel):
        self.a = acel
        self.vm = max(vmin, min(vmax, self.vm + self.a*pt))
        self.d =self.vm*pt


def iniciar(n): #con esta función se crean los agentes del modelo
    global vehiculos, filenames

    filenames = []

    vehiculos = []

    while len(vehiculos) < n*1:
        v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),1)
        if round(v.x,0) not in [round(z.x,0) for z in vehiculos]:
            vehiculos.append(v)


def parametros(v,vm):
  distancias = {
      ("vmax"): vmax,
      ("vm"): vm}
  return distancias.get((v),"NA")

def toroide(): #para el espacio
    for v in vehiculos:
        if v.x > tM:
            v.x = v.x % tM

def duplicar():
    global a,m
    for v in vehiculos:
      if v.x == min([v.x for v in vehiculos]):
        a = v.vm
        copia = Vehiculo("doble",min([v.x for v in vehiculos])+tM, a,1)
        vehiculos.append(copia)


def movimiento():
    vehiculos.sort(key = lambda v: v.x)
    for v,z in zip(vehiculos,vehiculos[1:]):
        d = z.x - v.x #distancia entre carros
        v.f = parametros(v.m, min(1,max(v.vm + 0.011,0.1)))
        if  d >= v.f*pt:
            v.mover(0.01+0.001*random.uniform(-1,1))
        else:
            v.vm = z.vm #con esto se toma la velocidad del siguiente carro
            v.mover(-0.02) # y con esto se frena
        v.x = v.d + v.x

def borrar():
    for v in vehiculos:
        if v.t == "doble":
            vehiculos.remove(v)

def imitacion():
  alrededor=[]
  for v in vehiculos:
    for z in vehiculos:
      if v!=z:
        if abs(v.x - z.x) <= 1.1:
          alrededor.append(z.m)
    if len(alrededor)!=0:
      if len([i for i in alrededor if i == "vmax"])>len([i for i in alrededor if i == "vm"]):
        v.m="vmax"
      elif len([i for i in alrededor if i == "vmax"])<len([i for i in alrededor if i == "vm"]):
        v.m="vm"
      else:
        pass
    alrededor=[]

def interaccion(n,s):
    global filenames
    for i in range(1,t+1):
        duplicar()
        imitacion()
        movimiento()
        borrar()
        toroide()

        #Información para gráficos por simulación
        vel1.append([n,st.mean([v.vm for v in vehiculos]),s])
        med1.append([n,st.median([v.vm for v in vehiculos]),s])
        des1.append([n,st.stdev([v.vm for v in vehiculos]),s])
        err1.append([n,ss.sem([v.vm for v in vehiculos]),s])



        #Para activar el GIF de carros se descomenta lo que viene a continuación:
        '''
        plt.figure(figsize=(20,3))
        plot([v.x for v in vehiculos if v.m=="vmax"], [v.y for v in vehiculos if v.m=="vmax"], "ro")
        plot([v.x for v in vehiculos if v.m=="vm"], [v.y for v in vehiculos if v.m=="vm"], "bo")
        #plot([v.x for v in vehiculos if v.id == 3], [v.y for v in vehiculos if v.id == 3], "yo", markersize = 10)
        text(tM/50,1.2,"Paso:")
        text(tM/20,1.2,i)
        axis([0,tM,0.8,1.2])

        filename = f'paso{i}.png'
        filenames.append(filename)
        plt.savefig(filename)
        plt.close()
        #'''



In [ ]:
#listas por paso
vel1 = []
med1 = []
des1 = []
err1 = []

for i in N:
  for j in range (1,s+1): #el s es la cantidad de simulaciones
    iniciar(i)
    interaccion(i,j)
  media=[]
  media.append(st.mean([l[1] for l in vel1 if l[0]==i]))
  print(media)
  print("-----------------------------","Acabó N =",i,"-----------------------------")


----------------------------- Acabó N = 5 -----------------------------
----------------------------- Acabó N = 10 -----------------------------
----------------------------- Acabó N = 15 -----------------------------
----------------------------- Acabó N = 20 -----------------------------
----------------------------- Acabó N = 25 -----------------------------
----------------------------- Acabó N = 30 -----------------------------
----------------------------- Acabó N = 35 -----------------------------
----------------------------- Acabó N = 40 -----------------------------
----------------------------- Acabó N = 45 -----------------------------


In [ ]:
#Código para generar el GIF, siempre y cuando se hayan guardado las imágenes por paso
gif_name= "Los 4 comportamientos"

with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        i = imageio.imread(filename)
        writer.append_data(i)

#Esto elimina automáticamente las imágenes por paso creadas
for filename in set(filenames):
    os.remove(filename)

#Types A, B and C

In [14]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss
import matplotlib.patches as patches

#Parámetros
N = [5,10,15,20,25,30,35,40,45] #cantidad de vehículos por experimento (densidad).
pt = 1 #paso temporal
s = 100 #cantidad de simulaciones por densidad
t = 500 #tiempo que dura cada simulación
vemax = 1 #máxima velocidad que puede tener un agente
vmin= 0 #mínima velocidad
tM= 50 #capacidad de agentes por carril y diámetro de espacio


class Vehiculo:
    id_iter = itertools.count() #para crearles un id a los carros
    def __init__(self, posx, velocidad, posy):
        self.id = next(Vehiculo.id_iter) #id de vehículo
        self.t = random.choice(["carro"]) #existen 3 tipos: carro (Type C), combi (Type C) y coaster (Type B)
        self.x = posx #para la posición del vehículo, coordenada horizontal
        self.y = posy #para la posición del vehículo, coordenada vertical
        self.vm = velocidad #velocidad del vehículo en el paso
        self.d = 0 #distancia recorrida por paso
        self.fd = random.choice(["vemax","vm"]) #etiqueta del valor considerado para comportamiento en la distancia de seguimiento. Para tener carros solo "vmax", eliminar "vm" de la lista y viceversa
        self.fdv = vemax #valor default considerado para comportamiento en la distancia de seguimiento

    def mover(self,acel): #función para la actualización de la distancia recorrida por un vehículo
        if self.t == "coaster":
          vmax = 2
        else:
          vmax = 1
        self.a = acel #parámetro recibido en la función, el cual es la aceleración.
        self.vm = max(vmin, min(vmax, self.vm + self.a*pt)) #actualización de la velocidad en base a la aceleración, máximo es 1, mínimo 0.
        self.d =self.vm*pt #actualización de la distancia avanzada en base a la velocidad del vehículo

class Carril:
    def __init__(self, posy):
        self.y = posy

def iniciar(n): #con esta función se crean los agentes del modelo
    global vehiculos, filenames, carriles, semafs

    filenames = []
    vehiculos = []
    carriles = []
    semafs=[]

    c1 = Carril(1)
    carriles.append(c1)

    while len(vehiculos) < n:
        v = Vehiculo(round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),1)
        if round(v.x,0) not in [round(z.x,0) for z in vehiculos]: #para agregar solo los que tienen una posición que no se ha ocupado
            vehiculos.append(v) #se añade el vehículo creado a la lista de vehículos cada uno con una posición diferente

def toroide(): #para simular un espacio toroidal
    for v in vehiculos:
        if v.x > tM: #si la posición está más allá del área del espacio, se da la "vuelta"
            v.x = v.x % tM

def duplicar(): #función para duplicar al agente que está al inicio del carril y ubicarlo adelante del que está al último para que el último sepa cómo moverse.
    global a
    for v in vehiculos:
      if v.x == min([v.x for v in vehiculos]):
        a = v.vm
        copia = Vehiculo(min([v.x for v in vehiculos])+tM,a,1)
        copia.t="doble"
        vehiculos.append(copia) #se añade el vehículo copiado temporalmente

def followingdistance(fd,vm): #función para que cada carro actualice su comportamiento en base a la etiqueta que se le asignó en la función iniciar()
  comportamientos = {
      ("vemax"): vemax, #distancia de seguimiento fija
      ("vm"): vm} #distancias de seguimiento variable
  return comportamientos.get((fd),"NA")


def distanciaseguimiento(): #función para actualización del valor considerado para el comportamiento en la distancia de seguimiento.
    for v in vehiculos:
        if v.t == "coaster":
            v.fdv = min(2,max(v.vm + 0.011,0.1))
        elif v.t == "combi":
            v.fdv = min(1,max(v.vm + 0.022,0.1))
        else:
            v.fdv = followingdistance(v.fd, min(1,max(v.vm + 0.011,0.1)))

def movimiento(): #función para que cada vehículo sepa cómo moverse.
    vehiculos.sort(key = lambda v: v.x)
    for v,z in zip(vehiculos,vehiculos[1:]):
        d = z.x - v.x #distancia entre vehículos
        distanciaseguimiento() #actualización del valor considerado para el comportamiento en la distancia de seguimiento.
        if  d >= v.fdv*pt: #si la distancia entre vehículos es mayor al valor de la distancia de seguimiento, acelera.
          if v.t == "combi":
            v.mover(0.02+0.002*random.uniform(-1,1))
          else:
            v.mover(0.01+0.001*random.uniform(-1,1)) #aumento de velocidad.
        else: #si es nenor, desacelera
            v.vm = z.vm #primero, iguala la velocidad del vehículo de adelante
            v.mover(-0.02) # y luego reduce su velocidad.
        v.x = v.d + v.x #se actualiza la nueva posición del vehículo en base a la distancia recorrida.

def borrar(): #función para borrar el duplicado creado
    for v in vehiculos:
        if v.t == "doble":
            vehiculos.remove(v)


def interaccion(n,s): #función que une las funciones que se necesitan en cada paso de la simulación
    global filenames
    for i in range(1,t+1): #se ejecuta el loop la cantidad de pasos (duración) que tiene cada simulación
      duplicar()
      movimiento()
      borrar()
      toroide()

      #Información recopilada por simulación para generar gráficos
      vel1.append([n,st.mean([v.vm for v in vehiculos]),s])
      med1.append([n,st.median([v.vm for v in vehiculos]),s])
      des1.append([n,st.stdev([v.vm for v in vehiculos]),s])
      err1.append([n,ss.sem([v.vm for v in vehiculos]),s])
      '''
      if len([v for v in vehiculos if v.m=="vm"])>1:
        vel1b.append([n,st.mean([v.vm for v in vehiculos if v.m=="vm"]),s])
        des1b.append([n,st.stdev([v.vm for v in vehiculos if v.m=="vm"]),s])
        med1b.append([n,st.median([v.vm for v in vehiculos if v.m=="vm"]),s])
        err1b.append([n,ss.sem([v.vm for v in vehiculos if v.m=="vm"]),s])
      if len([v for v in vehiculos if v.m=="vmax"])>1:
        med1a.append([n,st.median([v.vm for v in vehiculos if v.m=="vmax"]),s])
        des1a.append([n,st.stdev([v.vm for v in vehiculos if v.m=="vmax"]),s])
        vel1a.append([n,st.mean([v.vm for v in vehiculos if v.m=="vmax"]),s])
        err1a.append([n,ss.sem([v.vm for v in vehiculos if v.m=="vmax"]),s])#'''

      #Código para generar el GIF de una simulación. Para hacer el GIF, se descomenta lo que viene a continuación:
      '''
      ig, ax = plt.subplots(figsize=(25,25))
      #carros
      plot([v.x for v in vehiculos if v.m=="vmax"], [v.y for v in vehiculos if if v.m=="vmax"] ,'o', color="darkorange", markersize=3)
      plot([v.x for v in vehiculos if if v.m=="vm"], [v.y for v in vehiculos if v.m=="vm"] ,'o', color="deeppink", markersize=3)
      #carriles
      ax.add_patch(patches.Rectangle((0, 20.4),50,0.2,edgecolor = 'gray',facecolor = 'gray',fill=True))
      ax.add_patch(patches.Rectangle((0, 19.9),50,0.2,edgecolor = 'gray',facecolor = 'gray',fill=True))
      ax.add_patch(patches.Rectangle((29.9, 0),0.2,50,edgecolor = 'gray',facecolor = 'gray',fill=True))
      ax.add_patch(patches.Rectangle((30.4, 0),0.2,50,edgecolor = 'gray',facecolor = 'gray',fill=True))
      #cruce
      ax.add_patch(patches.Rectangle((29.95, 20.8),0.6,0.05,edgecolor = 'black',facecolor = 'yellow',fill=True))
      text(5,35,"Paso:")
      text(10,35,i)
      text(5,30,(n, "carros"))
      #text(5,28,("Periodo: ",pe))
      tick_params(labelcolor='w')
      #grid(b=True, axis='both', color = "white")
      axis([0,tM,0,tM])

      filename = f'paso{i}.png'
      filenames.append(filename)
      plt.savefig(filename)
      plt.close()
      #'''

In [15]:
#Este bloque ejecuta el modelo
#listas por paso
vel1 = []
med1 = []
des1 = []
err1 = []

for n in N: #para cada cantidad de carros (n) por simulación, se ejecuta s cantidad de simulaciones.
  for s in range (1,s+1): #el s es la cantidad de simulaciones
    iniciar(n)
    interaccion(n,s)
  media=[]
  media.append(st.mean([l[1] for l in vel1 if l[0]==n]))
  print(media)
  print("-----------------------------","Acabó N =",n,"-----------------------------")


[0.9330840237539647]
----------------------------- Acabó N = 5 -----------------------------
[0.9312126567073827]
----------------------------- Acabó N = 10 -----------------------------
[0.9262081848955095]
----------------------------- Acabó N = 15 -----------------------------
[0.9217404018341703]
----------------------------- Acabó N = 20 -----------------------------
[0.9082679123079587]
----------------------------- Acabó N = 25 -----------------------------
[0.585718870642506]
----------------------------- Acabó N = 30 -----------------------------
[0.4185104289122708]
----------------------------- Acabó N = 35 -----------------------------
[0.3389051895322218]
----------------------------- Acabó N = 40 -----------------------------
[0.26711082473703035]
----------------------------- Acabó N = 45 -----------------------------


In [ ]:
#Código para generar el GIF, siempre y cuando se hayan guardado las imágenes por paso
gif_name= "Los 4 comportamientos"

with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        i = imageio.imread(filename)
        writer.append_data(i)

#Esto elimina automáticamente las imágenes por paso creadas
for filename in set(filenames):
    os.remove(filename)